# Create a graph database in Neo4j for the BART system

Radhika Mardikar


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [2]:
driver = neo4j.GraphDatabase.driver(uri="", auth=("",""))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [7]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [8]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [9]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [10]:
connection = psycopg2.connect(
    user = "",
    password = "",
    host = "",
    port = "",
    database = "postgres"
)

In [11]:
cursor = connection.cursor()

# Wipe out the Neo4j database



In [25]:
my_neo4j_wipe_out_database()

# Verify the number of nodes and relationships



In [26]:
my_neo4j_number_nodes_relationships() 

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Find the list of stations and create the nodes in the graph

In [27]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

# Verify the number of nodes and relationships


In [28]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


# Find the list of stations, their lines, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with no weight


In [29]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    my_neo4j_create_node(line + ' ' + station)
    my_neo4j_create_relationship_one_way('depart ' + station, line + ' ' + station, 0)
    my_neo4j_create_relationship_one_way(line + ' ' + station, 'arrive ' + station, 0)
    



# Verify the number of nodes and relationships

In [30]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


# Find the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight


In [31]:
connection.rollback()

query = """

select l1.station, l1.line as from_line, l2.line as to_line, s.transfer_time
from lines as l1 
join lines as l2
on l1.station = l2.station and l1.line != l2.line

join stations as s
on s.station = l1.station

order by station, from_line, to_line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = float(row[3])
    

    my_neo4j_create_relationship_one_way(from_line + ' ' + station, to_line + ' ' + station, transfer_time)
    



# Verify the number of nodes and relationships

In [32]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


# Find the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways


In [33]:
connection.rollback()

query = """

select l1.line, l1.station as from_station, l2.station as to_station, t.travel_time as travel_time_in_seconds
from lines as l1 
join lines as l2
on l1.line = l2.line and (l1.sequence + 1 = l2.sequence)
join travel_times as t
on (t.station_1 = l1.station and t.station_2 = l2.station) or (t.station_2 = l1.station and t.station_1 = l2.station) 


order by line, from_station, to_station


"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time_in_seconds = float(row[3])
    

    my_neo4j_create_relationship_two_way(line + ' ' + from_station, line + ' ' + to_station, travel_time_in_seconds)
    



# Verify the number of nodes and relationships

In [34]:
my_neo4j_number_nodes_relationships() 

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
